In [1]:
# Dependencies
from bs4 import BeautifulSoup, NavigableString, Tag
import requests
import pandas as pd
from sqlalchemy import create_engine
# We also need to do a pip install geopy before running this code
from geopy.geocoders import Nominatim

In [2]:
# URL of page to be scraped
url = 'https://twistedsifter.com/2014/02/50-amazing-finds-on-google-earth/'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

# Initialize Nominatim API
geolocator = Nominatim(user_agent="MyApp")

def conversion(old):
    direction = {'N':1, 'S':-1, 'E': 1, 'W':-1}
    new = old.replace(u'°',' ').replace('\'',' ').replace('"',' ')
    new = new.split()
    new_dir = new.pop()
    new.extend([0,0,0])
    return (int(new[0])+int(new[1])/60.0+int(new[2])/3600.0) * direction[new_dir]


In [3]:
final_dataframe = []
# Retrieve the title divs
results = soup.find_all('div', align='center')
results

countries = ['USA', 'Canada', 'Northern Canada', 'Egypt', 'Tanzania', 'Australia', 'Austrlia', 'Argentina', 'China', 'Chile', 'UK', \
             'United Kingdom', 'Cyprus', 'The Netherlands', 'Hong Kong', 'Germany', 'Russia', 'France', 'Iraq', \
             'Niger', 'southern Ténéré of Niger', 'New Caledonia', 'Tanzania', 'Sudan', 'Japan']

images = []
# Retrieve the paragraphs
img_results = soup.find_all('p')
for img in img_results:
    # Retrieve the images of each paragraph
    img = img.find('img', class_='aligncenter')
    if img != None:
        #print(img['data-lazy-src'])
        images.append(img['data-lazy-src'])
        
counter = 0
for result in results:
    # scrape the article header 
    header = result.find('h3')
    if header != None:
        #print(header)
        header1 = header.text.split('/br')[0].split('\n')[0]
        if len(header.text.split('/br')[0].split('\n')) == 2:
            coordinates = ''
            location = header.text.split('/br')[0].split('\n')[1].split(', ')
        if len(header.text.split('/br')[0].split('\n')) >= 3:
            if len(header.text.split('/br')[0].split('\n')) == 3:
                coordinates = header.text.split('/br')[0].split('\n')[1]
                location = header.text.split('/br')[0].split('\n')[2].split(', ')
            if len(header.text.split('/br')[0].split('\n')) > 3:
                coordinates = header.text.split('/br')[0].split('\n')[2]
                location = header.text.split('/br')[0].split('\n')[3].split(', ')
        #print(location[len(location)-1])
        if location[len(location)-1].strip() in countries:
            country = location[len(location)-1].strip()
        else:
            country = 'USA'
        if country == 'Austrlia':
            country = 'Australia'
        if country == 'UK':
            country = 'United Kingdom'
        if country == 'southern Ténéré of Niger':
            country = 'Niger'
        if country == 'The Netherlands':
            country = 'Netherlands'
        if country == 'Northern Canada':
            country = 'Canada'
        
        if coordinates == '':
            location = geolocator.geocode(location[0] + ',' + location [1])
            coordinates = str(location.latitude) + ', ' + str(location.longitude)
        if header1 == '1. Airplane Boneyard':
            coordinates = "32°08’59.96″ N, 110°50’09.03″W"
        if '°' in coordinates:
            coordinates = coordinates.replace(u'’', "'")
            coordinates = coordinates.replace(u'″', '"')
            coordinates = coordinates.replace("N ", "N, ")
            coordinates = coordinates.replace('" N', '"N')
            coordinates = coordinates.replace("° N", "°N")
            coordinates = coordinates.replace("S ", "S, ")
            coordinates = coordinates.replace('" S', '"S')
            coordinates = coordinates.replace("° S", "°S")
            coordinates = coordinates.replace('" W', '"W')
            coordinates = coordinates.replace("° W", "°W")
            coordinates = coordinates.replace('" E', '"E')
            coordinates = coordinates.replace("° E", "°E")

            half_string = coordinates.find(",")
            if '"' in coordinates:
                start = coordinates.find(".", 0, half_string)
                stop = coordinates.find('"', 0, half_string)
                coordinates = coordinates[0: start:] + coordinates[stop::]
            
                start = coordinates.find(".", half_string, len(coordinates)-1)
                stop = coordinates.find('"', half_string, len(coordinates)-1)
                coordinates = coordinates[0: start:] + coordinates[stop::]
            else:
                start = coordinates.find(".", 0, half_string)
                stop = coordinates.find('°', 0, half_string)
                coordinates = coordinates[0: start:] + coordinates[stop::]
            
                start = coordinates.find(".", half_string, len(coordinates)-1)
                stop = coordinates.find('°', half_string, len(coordinates)-1)
                coordinates = coordinates[0: start:] + coordinates[stop::]
            
            lat, lon = coordinates.split(', ')
            coordinates = str(conversion(lat)) + ", " + str(conversion(lon))
        
        post = {
                'description': header1 + ', ' + images[counter],
                'country': country,
                'latitude': coordinates.split(',')[0],
                'longitude': coordinates.split(',')[1]
               }
            
        final_dataframe.append(post)
        
        #print(images[counter])
        #print(header1)
        #print(coordinates)
        #print(country)
        
        counter += 1

final_dataframe
final_df = pd.DataFrame(final_dataframe)

# Need to replace the name of the sqlite file
engine = create_engine('sqlite:///paranormal_activity.db')
# Need to replace the table name
final_df.to_sql(name='paranormal', con=engine, if_exists='append', index=False)
# Need to read from right table to verify insertion of data
pd.read_sql_query('select * from paranormal', con=engine).head()


,id,description,city,state,country,date,type,latitude,longitude,encounter_seconds,season
0,1,"1. Airplane Boneyard, https://twistedsifter.co...",None,None,USA,None,None,32.149722,-110.835833,None,None
1,2,"2. Mysterious Desert Pattern, https://twisteds...",None,None,Egypt,None,None,27.380556,33.631667,None,None
2,3,"3. Swastika-Shaped Building Complex, https://t...",None,None,USA,None,None,32.676111,-117.157500,None,None
3,4,"4. Buffalo Herd, https://twistedsifter.com/wp-...",None,None,Tanzania,None,None,-4.289167,31.396111,None,None
4,5,"5. Giant Triangle, https://twistedsifter.com/w...",None,None,USA,None,None,33.747252,-112.633853,None,None
